In [162]:
# Imports
import numpy as np
import pandas as pd
import pyodbc
from sklearn.feature_selection import chi2
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt

In [160]:
# Dataset Columns
data_cols = [ 'POSTED_SPEED_LIMIT',      'TRAFFICWAY_TYPE',             'CRASH_DAY_OF_WEEK'
             ,'TRAFFIC_CONTROL_DEVICE',  'ROADWAY_SURFACE_COND',        'CRASH_MONTH'
             ,'DEVICE_CONDITION',        'HIT_AND_RUN_I',               'ZIP_CODE_GROUP'
             ,'WEATHER_CONDITION',       'PRIM_CONTRIBUTORY_CAUSE',     'NUM_UNITS'
             ,'LIGHTING_CONDITION',      'SEC_CONTRIBUTORY_CAUSE',      'FIRST_CRASH_TYPE'
             ,'CRASH_HOUR_PERIOD']



data_cols = [ 'POSTED_SPEED_LIMIT',      'TRAFFICWAY_TYPE',             'CRASH_DAY_OF_WEEK'
             ,  'ROADWAY_SURFACE_COND',        'CRASH_MONTH'
             ,                  'ZIP_CODE_GROUP'
             ,'WEATHER_CONDITION'
             ,'LIGHTING_CONDITION'
             ,'CRASH_HOUR_PERIOD']


response_cols = ['CRASH_TYPE', 'DAMAGE', 'INJURY_CLASS', 'INJURIES_TOTAL']

all_cols = data_cols + response_cols

In [111]:
file_name = r'V:\_temp_ds\cda-project\03_Dataset - EDA\Desouky\crashes-cleaned-final.csv'
rows = pd.read_csv(file_name)

POSTED_SPEED_LIMIT      object
TRAFFICWAY_TYPE         object
CRASH_DAY_OF_WEEK       object
ROADWAY_SURFACE_COND    object
CRASH_MONTH             object
ZIP_CODE_GROUP          object
WEATHER_CONDITION       object
LIGHTING_CONDITION      object
CRASH_HOUR_PERIOD       object
dtype: object

In [161]:
# Database connection details
server = 'localhost'
database = '[chicago-crashes]'
username = 'sa'
password = 'zeft'

# Establish a connection
conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};Initial Catalog={database};UID={username};PWD={password};TrustServerCertificate=yes"
    # "DRIVER={ODBC Driver 17 for SQL Server};Server=127.0.0.1;Initial Catalog=[chicago-crashes];UID=sa;PWD=zeft;TrustServerCertificate=Yes"
)

# SQL query to retrieve data
query = """
SELECT """ + ','.join(all_cols) + """
FROM [chicago-crashes].[dbo].[crashes-cleaned-final]
"""

# Execute the query
cursor = conn.cursor()
cursor.execute(query)

# # Fetch all rows
rows = cursor.fetchall()

# Create a DataFrame from the retrieved data
data = pd.DataFrame(np.array(rows), columns=all_cols)

# Extract categorical columns
cat_columns = data_cols

# One-hot encode categorical data
ohe = OneHotEncoder()
onehot_data = ohe.fit_transform(data[cat_columns])
onehot_df = pd.DataFrame(onehot_data.toarray(), columns=ohe.get_feature_names_out(cat_columns))

# Evaluate feature importance using chi-squared
chi2_scores, _ = chi2(onehot_data, data[response_cols[0]])

# Select features based on scores (e.g., top-k features)
k = 10
selected_features = [f for f, _ in sorted(zip(cat_columns, chi2_scores), key=lambda x: x[1], reverse=True)[:k]]

d = {item: idx for idx, item in enumerate(data.columns)}
selected_feature_indices = [d.get(item) for item in selected_features]

# Build a model using selected features
X_train, X_test, y_train, y_test = train_test_split(onehot_data, data[response_cols[0]], test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train[:, selected_feature_indices], y_train)

# Evaluate model performance
accuracy = model.score(X_test[:, selected_feature_indices], y_test)
print(f"Model accuracy using selected features: {accuracy:.2f}")

y_pred = model.predict(X_test[:, selected_feature_indices])

confusion_matrix(y_test, y_pred)

Model accuracy using selected features: 0.73


array([[     0,  43667],
       [     0, 119902]], dtype=int64)

In [102]:
# Evaluate feature importance using chi-squared
chi2_scores, _ = chi2(onehot_data, data[response_cols[0]])

k = 20
selected_features = [f for f, _ in sorted(zip(cat_columns, chi2_scores), key=lambda x: x[1], reverse=True)[:k]]

selected_features

['CRASH_DAY_OF_WEEK',
 'WEATHER_CONDITION',
 'POSTED_SPEED_LIMIT',
 'TRAFFICWAY_TYPE',
 'LIGHTING_CONDITION',
 'ZIP_CODE_GROUP',
 'CRASH_MONTH',
 'CRASH_HOUR_PERIOD',
 'ROADWAY_SURFACE_COND']